In [ ]:
!pip install google-cloud-aiplatform google-auth langchain > /dev/null

Challenge: Want to reuse parts of prompts. 

Why Reuse: 
- To automate on a list
- Use it in application
- Use a prompt in different template

This can be done with a PipelinePrompt. A PipelinePrompt consists of two main parts:

final_prompt: This is the final prompt that is returned

pipeline_prompts: This is a list of tuples, consisting of a string (name) and a Prompt Template. Each PromptTemplate will be formatted and then passed to future prompt templates as a variable with the same name as name

In [42]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/content/generativeaitrial-trialLC.json'

In [43]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.llms import VertexAI

In [2]:
qa_maintemplate = """{introduction}

{example_qa}

{start}"""

qa_mainprompt = PromptTemplate.from_template(qa_maintemplate)

In [60]:
qa_mainprompt.input_variables

['example_qa', 'introduction', 'start']

In [3]:
introduction_qa_template = """You are impersonating {person}."""

introduction_qa_prompt = PromptTemplate.from_template(introduction_qa_template)

In [4]:
example_qa_template = """Here's an example of an interaction: 

Q: {example_q}
A: {example_a}"""
example_qa_prompt = PromptTemplate.from_template(example_qa_template)

In [5]:
start_qamain = """Now, do this for real!

Q: {input}
A:"""
start_qa_prompt = PromptTemplate.from_template(start_qamain)

In [6]:
input_qa_prompts = [
    ("introduction",introduction_qa_prompt),
    ("example_qa",example_qa_prompt),
    ("start",start_qa_prompt)
]

qa_pipeline_prompt = PipelinePromptTemplate(final_prompt=qa_mainprompt,
                                            pipeline_prompts=input_qa_prompts)

In [7]:
qa_pipeline_prompt.input_variables

['example_a', 'example_q', 'person', 'input']

Additional Example:::

In [8]:
task_maintemplate = """{task}

{example_task}

{start}"""

task_mainprompt = PromptTemplate.from_template(task_maintemplate)

In [29]:
introduction_task_template = """You are elite {character} given a {job}."""

introduction_task_prompt = PromptTemplate.from_template(introduction_task_template)

In [30]:
introduction_task_prompt.input_variables

['character', 'job']

In [24]:
example_task_template = """Here's an example of a task: 

task: {given_task}
soln: {given_soln}"""
example_task_prompt = PromptTemplate.from_template(example_task_template)

In [15]:
example_task_prompt.input_variables

['given_soln', 'given_task']

In [25]:
start_taskmain = """Now, get on with below task!

task: {input}
soln:"""
start_task_prompt = PromptTemplate.from_template(start_taskmain)

In [26]:
start_task_prompt.input_variables

['input']

In [31]:
input_task_prompts = [
    ("task",introduction_task_prompt),
    ("example_task",example_task_prompt),
    ("start",start_task_prompt)
]

task_pipeline_prompt = ''

task_pipeline_prompt = PipelinePromptTemplate(final_prompt=task_mainprompt,
                                            pipeline_prompts=input_task_prompts)

In [32]:
task_pipeline_prompt.input_variables

['character', 'job', 'given_task', 'given_soln', 'input']

In [ ]:
print(qa_pipeline_prompt.format(
    person="Neil Armstrong",
    example_q="What's your favorite Vehicle?",
    example_a="Rocket",
    input="What's your favorite social media site?"
))

You are impersonating Neil Armstrong.

Here's an example of an interaction: 

Q: What's your favorite Vehicle?
A: Rocket

Now, do this for real!

Q: What's your favorite social media site?
A:


In [35]:
print(task_pipeline_prompt.format(
    character="Math Wiz",
    job="code",
    given_task="""Write a function called reverse_string that takes a 
    string as input and returns the 
    reversed version of the string. """,
    given_soln="""def reverse_string(input_string):
    words = input_string.split(" ")  # Split the input string into a list of words
    reversed_words = []  # List to store reversed words

    for word in words:
        reversed_word = word[::-1]  # Reverse the characters in the word
        reversed_words.append(reversed_word)

    reversed_string = " ".join(reversed_words)  # Join the reversed words into a string

    return reversed_string

# Test the function
input_string = "Hello world"
print(reverse_string(input_string))  # Output: "olleH dlrow"
""",
    input="""Write a function is_palindrome(string) that takes in a 
    string as input and returns True or False"""
))

You are elite Math Wiz given a code.

Here's an example of a task: 

task: Write a function called reverse_string that takes a 
    string as input and returns the 
    reversed version of the string. 
soln: def reverse_string(input_string):
    words = input_string.split(" ")  # Split the input string into a list of words
    reversed_words = []  # List to store reversed words

    for word in words:
        reversed_word = word[::-1]  # Reverse the characters in the word
        reversed_words.append(reversed_word)

    reversed_string = " ".join(reversed_words)  # Join the reversed words into a string

    return reversed_string

# Test the function
input_string = "Hello world"
print(reverse_string(input_string))  # Output: "olleH dlrow"


Now, get on with below task!

task: Write a function is_palindrome(string) that takes in a 
    string as input and returns True or False
soln:


3rd Example

In [40]:
input_composed_prompts = [
    ("task",introduction_qa_prompt),
    ("example_task",example_task_prompt),
    ("start",start_task_prompt)
]
task_composed_prompt = ''
task_composed_prompt = PipelinePromptTemplate(final_prompt=task_mainprompt,
                                            pipeline_prompts=input_composed_prompts)


In [41]:
task_composed_prompt.input_variables

['given_task', 'person', 'given_soln', 'input']

In [52]:
from langchain import PromptTemplate, LLMChain

llm = VertexAI(temperature=0)

llm_chain = LLMChain(
    llm=llm,
    prompt=qa_pipeline_prompt
)

In [53]:
llm_chain.input_keys

['example_a', 'example_q', 'person', 'input']

In [56]:
llm_chain({"input":'When is your Birth Day',
          "person":'Obama',
          "example_q":'When did you study in Harvard?',
          "example_a": 1991})

{'input': 'When is your Birth Day',
 'person': 'Obama',
 'example_q': 'When did you study in Harvard?',
 'example_a': 1991,
 'text': 'August 4, 1961'}

In [59]:
llm_chain({"input":'How was it to study in Harvard',
          "person":'Henry Kissinger',
          "example_q":'When did you study in Harvard?',
          "example_a": 1950})

{'input': 'How was it to study in Harvard',
 'person': 'Henry Kissinger',
 'example_q': 'When did you study in Harvard?',
 'example_a': 1950,
 'text': 'I was a very good student.'}

In [ ]:
llm_chain = LLMChain(
    llm=llm,
    prompt=qa_pipeline_prompt
)